libraries

In [1]:
from pyomo.environ import *
from brooks import *
import pandas as pd

In [2]:
sample_data = pd.read_excel("Sample data.xlsx", 
                          sheet_name="sample_data")
team_data = pd.read_excel("Sample data.xlsx", 
                          sheet_name="Team_data")
exp_data = pd.read_excel("Sample data.xlsx", 
                          sheet_name="Expertise_data")
data = pd.read_excel("Sample data.xlsx", 
                          sheet_name="data")
data_2 = pd.read_excel("Sample data.xlsx", 
                          sheet_name="data2")


W = list(sample_data["Worker_ID"])
T = list(team_data["Teams"])
F = list(data_2["Function"])
E =list(exp_data["Expertise"])
P =list(data["A.Experience"])
Z =list(data["Time_zone"])
R =list(data_2["years_of_experience"])
C = list(data_2["Experience"])
###
teams_max = 20
function = 3
agile_cost = 2286
time_cost = 200
exp_cost=100
product_cost = 50
et_cost = 80
member=10
fs =1
fm =1
ts=2
tm=2

defining Model

In [3]:
model = Model()

defining variables

In [4]:
if hasattr(model,'x'):
    model.delete(model.x)
model.x = Var(W,T, domain = Binary)

In [5]:
if hasattr(model,'y'):
    model.delete(model.y)
model.y = Var(T,Z, domain = Binary)

In [6]:
if hasattr(model,'z'):
    model.delete(model.z)
model.z = Var(T, domain = Binary )

In [7]:
if hasattr(model,'c'):
    model.delete(model.c)
model.c = Var(T,C, domain = NonNegativeIntegers)

In [8]:
if hasattr(model,'f'):
    model.delete(model.f)
model.f = Var(T,F, domain = NonNegativeIntegers)

In [9]:
if hasattr(model,'a'):
    model.delete(model.a)
model.a = Var(T, domain = Binary )

In [10]:
if hasattr(model,'ae'):
    model.delete(model.ae)
model.ae = Var(T, domain = NonNegativeIntegers )

In [11]:
if hasattr(model,'ts'):
    model.delete(model.ts)
model.ts = Var(T, domain = NonNegativeReals, bounds = (0,2))

In [12]:
if hasattr(model,'tm'):
    model.delete(model.tm)
model.tm = Var(T, domain = NonNegativeReals, bounds = (0,2))

In [13]:
if hasattr(model,'fs'):
    model.delete(model.fs)
model.fs = Var(T,C,domain = NonNegativeIntegers, bounds = (0,1))

In [14]:
if hasattr(model,'fm'):
    model.delete(model.fm)
model.fm = Var(T,C,domain = NonNegativeIntegers, bounds = (0,1))

In [15]:
if hasattr(model,'fps'):
    model.delete(model.fpplus)
model.fpplus = Var(T,F,domain = NonNegativeIntegers, bounds = (0,1))

In [16]:
if hasattr(model,'fpm'):
    model.delete(model.fpm)
model.fpm = Var(T,F,domain = NonNegativeIntegers, bounds = (0,1))

 Define Objective Function 

In [17]:
if hasattr(model, 'cost_objective'):
    model.delete(model.cost_objective)

model.cost_objective = Objective(expr = (agile_cost*sum(model.z[j]  for j in T))+ (time_cost*sum(model.y[j,k] for j in T for k in Z)) + (exp_cost*sum(model.fs[j,k] + model.fm[j,k] for j in T for k in C))+ (product_cost*sum(model.x[i,j] for i in W for j in T ))+ (et_cost*sum(model.x[i,j] for i in W for j in T )) ,sense=minimize)


Defining Constraints

In [18]:

if hasattr(model, 'allocation_constraint'):
    model.delete(model.allocation_constraint)
model.allocation_constraint = ConstraintList()
for i in W:
    if E != "Scrum Master":
        model.allocation_constraint.add(sum(model.x[i,j] for j in T) == 1)


In [19]:

if hasattr(model, 'teamsize_constraint'):
    model.delete(model.teamsize_constraint)
model.teamsize_constraint = ConstraintList()
for j in T:
    model.teamsize_constraint.add(sum(model.x[i,j] for i in W) <= (10+ts-tm))

In [20]:

if hasattr(model, 'Expertise_constraint'):
    model.delete(model.Expertise_constraint)
model.Expertise_constraint = ConstraintList()
for k in C:
    model.Expertise_constraint.add(sum(model.c[j,k] for j in T) <= (2.5+ fs -fm))

In [21]:

if hasattr(model,'owner_constraint'):
    model.delete(model.owner_constraint)
if F == "R":
    model.owner_constraint = ConstraintList()
    for j in T:
        model.owner_constraint.add(sum(model.x[i,j] for i in W) >= 1)


In [22]:

if hasattr(model, 'functional_constraint'):
    model.delete(model.functional_constraint)
model.functional_constraint = ConstraintList()
for j in T:
    for k in F:
        if k != "Scrum":
            if F == k:
                model.functional_constraint.add(sum(model.x[i,j] for i in W  ) == function+model.fps[j,k]-model.fpm[j,k])

In [23]:

if hasattr(model, 'agile_constraint'):
    model.delete(model.agile_constraint)
model.agile_constraint = ConstraintList()
for j in T:
    for i in W:
        if P == "Hands-on experience":
            model.agile_constraint.add(sum(model.x[i,j])>= model.a[j])

In [24]:
if hasattr(model, 'need_constraint'):
    model.delete(model.need_constraint)
model.need_constraint= ConstraintList()
for j in T:
    for i in W:
        if P == "Hands-on experience":
             model.need_constraint.add(sum(model.x[i,j]) - 0.2* sum(model.x[i,j] for i in W)) >= -10*(1-model.ae[j])
    


Using gurobi solver

In [25]:
solver = SolverFactory('gurobi',solver_io="python")
status = solver.solve(model)

In [26]:
print('status = %s' % status)

status = 
Problem: 
- Name: unknown
  Lower bound: 26000.0
  Upper bound: 26000.0
  Number of objectives: 1
  Number of constraints: 224
  Number of variables: 4640
  Number of binary variables: 4100
  Number of integer variables: 4600
  Number of continuous variables: -4060
  Number of nonzeros: 8080
  Sense: 1
  Number of solutions: 1
Solver: 
- Name: Gurobi 9.51
  Status: ok
  Wallclock time: 0.05953407287597656
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



printing result

In [27]:
for i in W:
    for j in T:
        if value(model.x[i,j])>0:
            print('%s =%f' % (model.x[i,j], value(model.x[i,j])))
print("objective = %f" % value(model.cost_objective))

x[101,19] =1.000000
x[102,19] =1.000000
x[103,20] =1.000000
x[104,12] =1.000000
x[105,14] =1.000000
x[106,5] =1.000000
x[107,6] =1.000000
x[108,1] =1.000000
x[109,4] =1.000000
x[110,9] =1.000000
x[111,10] =1.000000
x[112,16] =1.000000
x[113,15] =1.000000
x[114,11] =1.000000
x[115,11] =1.000000
x[116,2] =1.000000
x[117,9] =1.000000
x[118,11] =1.000000
x[119,15] =1.000000
x[120,5] =1.000000
x[121,10] =1.000000
x[122,15] =1.000000
x[123,10] =1.000000
x[124,6] =1.000000
x[125,4] =1.000000
x[126,3] =1.000000
x[127,3] =1.000000
x[128,13] =1.000000
x[129,7] =1.000000
x[130,12] =1.000000
x[131,11] =1.000000
x[132,11] =1.000000
x[133,18] =1.000000
x[134,18] =1.000000
x[135,16] =1.000000
x[136,10] =1.000000
x[137,1] =1.000000
x[138,5] =1.000000
x[139,8] =1.000000
x[140,18] =1.000000
x[141,9] =1.000000
x[142,8] =1.000000
x[143,13] =1.000000
x[144,8] =1.000000
x[145,4] =1.000000
x[146,2] =1.000000
x[147,11] =1.000000
x[148,2] =1.000000
x[149,10] =1.000000
x[150,4] =1.000000
x[151,4] =1.000000
x[15